In [1]:
# Imports

import time, math
import numpy as np
from enum import Enum, auto

In [ ]:
# Global Variables

max_weight = 5

class Deadlines(Enum):
    no_deadline = auto()

In [5]:
class Task:
    maqsad = True
    def __init__(self, name) -> None:
        self.name = name
        self.description = ""

        self.weight = 1

    def show_maqsad(self):
        print(self.maqsad)

In [8]:
task1 = Task("Complete CS201 Quiz 1")
task1.show_maqsad()
task1.maqsad = False
task1.show_maqsad()

True
False
